このノートブックでは、
染色体ごとに保存されているゲノムデータ (VCF形式)をマージし、matrix table 形式で保存します。

その後、imputation quality の低いバリアント (DR2 < 0.3) を除外します。

またその後のゲノムデータを用いて、PRS の計算手順を説明します。

2種類の脳梗塞 PRS モデルを事例として用います。

- PGS002724 モデルに含まれるバリアント数 1213574個
- PGS002725 モデルに含まれるバリアント数 6010730個


## このノートブックを実行する前に下記をご注意ください

- このノートブック用の jupyter サービスを立ち上げる前に `export PYSPARK_SUBMIT_ARGS='--driver-memory 48g --executor-memory 48g pyspark-shell'` のようにこの計算用のメモリを大きく確保してください。(この場合48ギガのメモリを割り当てています。) これを行っていただかないと、いずれかのセルで OutOfMemory エラーが発生し、それ以降のセル実行は機能しなくなります。
- hail の背後では spark が働いており、普段の jupyter notebook のようにセル実行を停止しても、その計算は動き続けます。その状態で新たなセルを実行すると予期せぬエラーが発生する場合があります。そのため「セルの実行がなかなか終わらないな」とお思いになられても、実行状態が完了するまで停止操作は行わないことをおすすめします。
- 本ノートブックは https://github.com/hacchy1983/prs-on-hail-public に変更を加えたものになります。


## Step 1 hail など、必要なモジュールを読み込みます
下記のコードを実行してください。 ページ上側のメニューバーにある 実行 ボタンを押下することで、実行することができます。

In [1]:
import hail as hl
hl.init()
from hail.plot import show
from pprint import pprint
hl.plot.output_notebook()

2022-11-23 12:14:48.075 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://guaca021:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.105-acd89e80c345
LOGGING: writing to /lustre8/home/kozonishida-pg/prs-on-hail/hail-20221123-1214-0.2.105-acd89e80c345.log


Loading BokehJS ...

In [57]:
!ls outputs/

chr1.beagle.log		 chr20.conform-gt.vcf.gz
chr1.beagle.vcf.gz	 chr20.mt
chr1.beagle.vcf.gz.tbi	 chr21.beagle.log
chr1.conform-gt.log	 chr21.beagle.vcf.gz
chr1.conform-gt.vcf.gz	 chr21.beagle.vcf.gz.tbi
chr1.mt			 chr21.conform-gt.log
chr10.beagle.log	 chr21.conform-gt.vcf.gz
chr10.beagle.vcf.gz	 chr21.mt
chr10.beagle.vcf.gz.tbi  chr22.beagle.log
chr10.conform-gt.log	 chr22.beagle.vcf.gz
chr10.conform-gt.vcf.gz  chr22.beagle.vcf.gz.tbi
chr10.mt		 chr22.conform-gt.log
chr11.beagle.log	 chr22.conform-gt.vcf.gz
chr11.beagle.vcf.gz	 chr22.mt
chr11.beagle.vcf.gz.tbi  chr3.beagle.log
chr11.conform-gt.log	 chr3.beagle.vcf.gz
chr11.conform-gt.vcf.gz  chr3.beagle.vcf.gz.tbi
chr11.mt		 chr3.conform-gt.log
chr12.beagle.log	 chr3.conform-gt.vcf.gz
chr12.beagle.vcf.gz	 chr3.mt
chr12.beagle.vcf.gz.tbi  chr4.beagle.log
chr12.conform-gt.log	 chr4.beagle.vcf.gz
chr12.conform-gt.vcf.gz  chr4.beagle.vcf.gz.tbi
chr12.mt		 chr4.conform-gt.log
chr13.beagle.log	 chr4.conform-gt.vcf.gz
chr13.beagle.vcf.gz	 chr4

## Step 2 ゲノムデータのファイル形式を変換します

ゲノムデータは、1番染色体から22番染色体まで、染色体ごとに異なるファイルに保存されています。
例えば、1番染色体のゲノムデータは、 `outputs/chr1.beagle.vcf.gz` に保存されています。
また、2番染色体のゲノムデータは、 `outputs/chr2.beagle.vcf.gz` に 22番染色体のゲノムデータは、 `outputs/chr22.beagle.vcf.gz` に保存されています。

ゲノムデータは、よく利用されるファイル形式である VCF フォーマットで保存されています。
これを、matrix table 形式に変換して、保存します。

下記コマンドを実行してください。
このセルの実行にはかなりの待ち時間が生じますが「このノートブックを実行する前に下記をご注意ください」のように途中でセル実行を停止することはおすすめしません。

In [58]:
for chr in range(1,23):
    infile = 'outputs/chr' + str(chr) + '.beagle.vcf.gz'
    outfile = 'outputs/chr' + str(chr) + '.mt'
    print(chr, infile, outfile)
    hl.import_vcf(infile, force_bgz=True).write(outfile, overwrite=True)

1 outputs/chr1.beagle.vcf.gz outputs/chr1.mt


2022-11-24 00:12:03.759 Hail: INFO: scanning VCF for sortedness...
2022-11-24 00:12:36.902 Hail: INFO: Coerced sorted VCF - no additional import work to do
2022-11-24 00:26:13.044 Hail: INFO: wrote matrix table with 2428653 rows and 2318 columns in 7 partitions to outputs/chr1.mt


2 outputs/chr2.beagle.vcf.gz outputs/chr2.mt


2022-11-24 00:26:13.449 Hail: INFO: scanning VCF for sortedness...
2022-11-24 00:26:45.883 Hail: INFO: Coerced sorted VCF - no additional import work to do
2022-11-24 00:41:24.089 Hail: INFO: wrote matrix table with 2627240 rows and 2318 columns in 7 partitions to outputs/chr2.mt


3 outputs/chr3.beagle.vcf.gz outputs/chr3.mt


...

2022-11-24 04:37:34.601 Hail: INFO: scanning VCF for sortedness...
2022-11-24 04:37:55.931 Hail: INFO: Coerced sorted VCF - no additional import work to do
2022-11-24 04:51:29.869 Hail: INFO: wrote matrix table with 679241 rows and 2318 columns in 2 partitions to outputs/chr20.mt


21 outputs/chr21.beagle.vcf.gz outputs/chr21.mt


2022-11-24 04:51:30.363 Hail: INFO: scanning VCF for sortedness...
2022-11-24 04:52:11.583 Hail: INFO: Coerced sorted VCF - no additional import work to do
2022-11-24 05:09:18.461 Hail: INFO: wrote matrix table with 427409 rows and 2318 columns in 1 partition to outputs/chr21.mt


22 outputs/chr22.beagle.vcf.gz outputs/chr22.mt


2022-11-24 05:09:18.993 Hail: INFO: scanning VCF for sortedness...
2022-11-24 05:09:54.515 Hail: INFO: Coerced sorted VCF - no additional import work to do
2022-11-24 05:26:20.144 Hail: INFO: wrote matrix table with 424147 rows and 2318 columns in 1 partition to outputs/chr22.mt


## Step 3 ゲノムデータをマージします
続いて、1番染色体から22番染色体の matrix table 形式のファイルを、ひとつの matrix table 形式のファイルにマージします。

下記コマンドを実行してください。

In [8]:
chr = 1
file = 'outputs/chr' + str(chr) + '.mt'
mt = hl.read_matrix_table(file)
print(chr, file, mt.count(), mt.count())
for chr in range(2,23):
    file = 'outputs/chr' + str(chr) + '.mt'
    tmpmt = hl.read_matrix_table(file)
    mt = mt.union_rows(tmpmt)
    print(chr, file, tmpmt.count(), mt.count())

1 outputs/chr1.mt (2428653, 2318) (2428653, 2318)
2 outputs/chr2.mt (2627240, 2318) (5055893, 2318)
3 outputs/chr3.mt (2186425, 2318) (7242318, 2318)
4 outputs/chr4.mt (2212857, 2318) (9455175, 2318)
5 outputs/chr5.mt (1986979, 2318) (11442154, 2318)
6 outputs/chr6.mt (1964598, 2318) (13406752, 2318)
7 outputs/chr7.mt (1801231, 2318) (15207983, 2318)
8 outputs/chr8.mt (1722793, 2318) (16930776, 2318)
9 outputs/chr9.mt (1342561, 2318) (18273337, 2318)
10 outputs/chr10.mt (1532460, 2318) (19805797, 2318)
11 outputs/chr11.mt (1520309, 2318) (21326106, 2318)
12 outputs/chr12.mt (1467858, 2318) (22793964, 2318)
13 outputs/chr13.mt (1099285, 2318) (23893249, 2318)
14 outputs/chr14.mt (1002655, 2318) (24895904, 2318)
15 outputs/chr15.mt (911380, 2318) (25807284, 2318)
16 outputs/chr16.mt (988654, 2318) (26795938, 2318)
17 outputs/chr17.mt (864311, 2318) (27660249, 2318)
18 outputs/chr18.mt (864327, 2318) (28524576, 2318)
19 outputs/chr19.mt (706126, 2318) (29230702, 2318)
20 outputs/chr20.mt 

ゲノムデータのマージが完了しました。
マージ後のゲノムデータのバリアント数を表示します。

In [9]:
mt.count()

(30761499, 2318)

これは、次のことを意味します。

- 研究対象者の人数が 2318 名
- バリアントの個数が 30761499 個

## Step 4 マージしたゲノムデータを保存します
今後のステップの実行時間を短縮するため、マージしたゲノムデータを保存し、再度読み込みます。

下記のコードは、マージしたゲノムデータを outputs/chrAll.mt に保存します。


In [10]:
mt.write('outputs/chrAll.mt', overwrite=True)

2022-11-15 20:03:09.905 Hail: INFO: wrote matrix table with 30761499 rows and 2318 columns in 89 partitions to outputs/chrAll.mt
    Total size: 18.99 GiB
    * Rows/entries: 18.99 GiB
    * Columns: 9.09 KiB
    * Globals: 11.00 B
    * Smallest partition: 282779 rows (181.07 MiB)
    * Largest partition:  424147 rows (311.72 MiB)


下記のコードは、 `outputs/chrAll.mt` を読み込みます。

In [2]:
mt = hl.read_matrix_table('outputs/chrAll.mt')

## Step 5 ゲノムデータに variantID を追加します
下記のコードは、ゲノムデータのバリアント情報に `variantID` を追加します。

In [3]:
mt = mt.annotate_rows(variantID = (hl.str(mt.locus.contig) + ":" + hl.str(mt.locus.position)) )

下記のコードは、 `variantID` を追加した後のバリアント情報を表示します。
`show(5)` は、先頭の 5 個のバリアントのみを表示する、ことを意味します。

In [4]:
mt.rows().show(5)

+---------------+--------------------------------+---------------+-----------+
| locus         | alleles                        | rsid          |      qual |
+---------------+--------------------------------+---------------+-----------+
| locus<GRCh37> | array<str>                     | str           |   float64 |
+---------------+--------------------------------+---------------+-----------+
| 1:10177       | ["A","AC"]                     | "rs367896724" | -1.00e+01 |
| 1:10235       | ["T","TA"]                     | "rs540431307" | -1.00e+01 |
| 1:10352       | ["T","TA"]                     | "rs555500075" | -1.00e+01 |
| 1:10616       | ["CCGCCGTTGCAAAGGCGCGCCG","C"] | "rs376342519" | -1.00e+01 |
| 1:10642       | ["G","A"]                      | "rs558604819" | -1.00e+01 |
+---------------+--------------------------------+---------------+-----------+

+----------+----------------+----------------+----------+-----------+
| filters  | info.AF        | info.DR2       | info.IMP | variantID |
+----------+----------------+----------------+----------+-----------+
| set<str> | array<float64> | array<float64> |     bool | str       |
+----------+----------------+----------------+----------+-----------+
| {}       | [4.46e-01]     | [0.00e+00]     |     True | "1:10177" |
| {}       | [4.00e-04]     | [0.00e+00]     |     True | "1:10235" |
| {}       | [4.73e-01]     | [0.00e+00]     |     True | "1:10352" |
| {}       | [9.93e-01]     | [0.00e+00]     |     True | "1:10616" |
| {}       | [3.70e-03]     | [0.00e+00]     |     True | "1:10642" |
+----------+----------------+----------------+----------+-----------+
showing top 5 rows

`variantID` のカラムが追加されていることが分かります。

## Step 6 imputation quality に基づいてゲノムデータをフィルタリングします

mt には multi-allelic site が含まれます。次にその割合を確認してみます。

In [5]:
mt1 = mt.filter_rows(hl.len(mt.info.DR2) == 1)
mt1.count()

(30361461, 2318)

In [6]:
mtnot1 = mt.filter_rows(hl.len(mt.info.DR2) > 1)
mtnot1.count()

(400038, 2318)

1% ほどが multi-allelic site であることがわかります。 1% は無視するにはやや多いですが、
このチュートリアルでは内容をわかりやすくするために除外したもの(すなわち`mt1`)を今後用います。
下記のコードでは、各バリアントの `imputation quality` の分布を表示します。 

In [7]:
p = hl.plot.histogram(mt1.info.DR2.first(), title='Imputation Quality Histogram', legend='Imputation Quality (DR2)')
show(p)

`imputation quality` が低いバリアントが少しあることが分かります。
下記のコードは、`imputation quality` が低い（DR2 < 0.3）バリアントを除外します。

In [8]:
mt1_filt = mt1.filter_rows(mt1.info.DR2.first()>=0.3)

下記のコードは、`imputation quality` が低いバリアントを除外した後の分布を表示します。

In [12]:
p = hl.plot.histogram(mt1_filt.info.DR2.first(), title='Imputation Quality Histogram', legend='Imputation Quality (DR2)')
show(p)

`imputation quality` が低いバリアントが除外されたことが分かります。

下記のコードは、`imputation quality` が低いバリアントを除外した後のバリアントの個数を表示します。

In [10]:
mt1_filt.count()

(29799034, 2318)

(29799034, 2318) と表示されました。

これは、次のことを意味します。

研究対象者の人数が 2318 名
バリアントの個数が 29799034 個
下記のコードは、`imputation quality` が低いバリアントを除外した後の allele frequency の分布を表示します。

In [11]:
p = hl.plot.histogram(mt1_filt.info.AF.first(), title='AF Histogram', legend='AF', bins=50)
show(p)

AF<1% のバリアントが多くあることが分かります。

## Step 7 フィルタリング後のゲノムデータを保存します
下記のコードは、マージしたゲノムデータを `outputs/chrAll.filtered.mt` に保存します。
ここまでのセルには実行完了までに時間がかかるものが存在します。
ここをチェックポイントとして同じことに時間をかけずに再開できるようにしておきましょう。

In [12]:
mt1_filt.write('outputs/chrAll.filtered.mt', overwrite=True)

2022-11-17 18:33:43.273 Hail: INFO: wrote matrix table with 29799034 rows and 2318 columns in 89 partitions to outputs/chrAll.filtered.mt


## Step 8 ゲノムデータを読み込みます
下記のコードでは、Step 7 で保存したゲノムデータを読み込みます。

In [ ]:
mt1_filt = hl.read_matrix_table('outputs/chrAll.filtered.mt')

In [67]:
!ls -alh outputs

合計 14G
drwxr-xr-x 26 kozonishida-pg oo-nig-pg  12K 11月 17 14:14 .
drwxr-xr-x  5 kozonishida-pg oo-nig-pg 4.0K 11月 17 16:56 ..
-rw-r--r--  1 kozonishida-pg oo-nig-pg 6.7K 11月 14 14:44 chr1.beagle.log
-rw-r--r--  1 kozonishida-pg oo-nig-pg 914M 11月 14 14:44 chr1.beagle.vcf.gz
-rw-r--r--  1 kozonishida-pg oo-nig-pg 207K 11月 14 14:44 chr1.beagle.vcf.gz.tbi
-rw-r--r--  1 kozonishida-pg oo-nig-pg  16M 11月 14 14:46 chr1.conform-gt.log
-rw-r--r--  1 kozonishida-pg oo-nig-pg  88M 11月 14 14:46 chr1.conform-gt.vcf.gz
drwxr-xr-x  8 kozonishida-pg oo-nig-pg 4.0K 11月 14 15:18 chr1.mt
-rw-r--r--  1 kozonishida-pg oo-nig-pg 4.5K 11月 14 14:44 chr10.beagle.log
-rw-r--r--  1 kozonishida-pg oo-nig-pg 574M 11月 14 14:45 chr10.beagle.vcf.gz
-rw-r--r--  1 kozonishida-pg oo-nig-pg 121K 11月 14 14:44 chr10.beagle.vcf.gz.tbi
-rw-r--r--  1 kozonishida-pg oo-nig-pg  10M 11月 14 14:46 chr10.conform-gt.log
-rw-r--r--  1 kozonishida-pg oo-nig-pg  59M 11月 14 14:46 chr10.conform-gt.vcf.gz
drwxr-xr-x  8 kozonishida-pg oo-

drwxr-xr-x  8 kozonishida-pg oo-nig-pg 4.0K 11月 14 16:46 chr8.mt
-rw-r--r--  1 kozonishida-pg oo-nig-pg 4.5K 11月 14 14:44 chr9.beagle.log
-rw-r--r--  1 kozonishida-pg oo-nig-pg 496M 11月 14 14:45 chr9.beagle.vcf.gz
-rw-r--r--  1 kozonishida-pg oo-nig-pg 110K 11月 14 14:44 chr9.beagle.vcf.gz.tbi
-rw-r--r--  1 kozonishida-pg oo-nig-pg 8.5M 11月 14 14:46 chr9.conform-gt.log
-rw-r--r--  1 kozonishida-pg oo-nig-pg  52M 11月 14 14:46 chr9.conform-gt.vcf.gz
drwxr-xr-x  8 kozonishida-pg oo-nig-pg 4.0K 11月 14 16:56 chr9.mt
drwxr-xr-x  8 kozonishida-pg oo-nig-pg 4.0K 11月 17 15:29 chrAll.filtered.mt
drwxr-xr-x  8 kozonishida-pg oo-nig-pg 4.0K 11月 15 20:03 chrAll.mt
-rw-r--r--  1 kozonishida-pg oo-nig-pg 1.8K 11月 14 14:44 chrX_PAR1.beagle.log
-rw-r--r--  1 kozonishida-pg oo-nig-pg  43M 11月 14 14:45 chrX_PAR1.beagle.vcf.gz
-rw-r--r--  1 kozonishida-pg oo-nig-pg 1.8K 11月 14 14:44 chrX_PAR1.beagle.vcf.gz.tbi
-rw-r--r--  1 kozonishida-pg oo-nig-pg 123K 11月 14 14:46 chrX_PAR1.conform-gt.log
-rw-r--r--  1 k

## Step 9-1 PRSモデルを読み込みます (PGS002724, PGS002725)
下記のコードを実行すると、`prs-models/PGS002724.txt` と `prs-models/PGS002724.txt` のデータが読み込まれます。

In [34]:
!wget https://ftp.ebi.ac.uk/pub/databases/spot/pgs/scores/PGS002724/ScoringFiles/PGS002724.txt.gz

--2022-11-17 14:37:13--  https://ftp.ebi.ac.uk/pub/databases/spot/pgs/scores/PGS002724/ScoringFiles/PGS002724.txt.gz
ftp.ebi.ac.uk (ftp.ebi.ac.uk) をDNSに問いあわせています... 193.62.193.138
ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.138|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 16818717 (16M) [application/x-gzip]
`PGS002724.txt.gz' に保存中

PGS002724.txt.gz    100%[===================>]  16.04M  7.16MB/s    in 2.2s    

2022-11-17 14:37:16 (7.16 MB/s) - `PGS002724.txt.gz' へ保存完了 [16818717/16818717]



In [35]:
!wget https://ftp.ebi.ac.uk/pub/databases/spot/pgs/scores/PGS002725/ScoringFiles/PGS002725.txt.gz

--2022-11-17 14:37:19--  https://ftp.ebi.ac.uk/pub/databases/spot/pgs/scores/PGS002725/ScoringFiles/PGS002725.txt.gz
ftp.ebi.ac.uk (ftp.ebi.ac.uk) をDNSに問いあわせています... 193.62.193.138
ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.138|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 81992802 (78M) [application/x-gzip]
`PGS002725.txt.gz' に保存中

PGS002725.txt.gz     99%[==================> ]  77.85M  1009KB/s    eta 0s     


PGS002725.txt.gz     99%[==================> ]  78.12M  1.02MB/s    eta 0s     
PGS002725.txt.gz    100%[===================>]  78.19M  1.04MB/s    in 58s     

2022-11-17 14:38:18 (1.34 MB/s) - `PGS002725.txt.gz' へ保存完了 [81992802/81992802]



In [36]:
!gunzip PGS002724.txt.gz

In [37]:
!gunzip PGS002725.txt.gz

In [38]:
!ls

PGS002724.txt
PGS002725.txt
Tutorial_2022_11_15.ipynb
hail-20221111-1605-0.2.105-acd89e80c345.log
hail-20221114-1432-0.2.105-acd89e80c345.log
outputs
prs-models


In [28]:
!mkdir prs-models

In [39]:
!mv PGS002724.txt prs-models/

In [40]:
!mv PGS002725.txt prs-models/

In [13]:
!ls prs-models

PGS002724.txt  PGS002725.txt


In [15]:
!head prs-models/PGS002724.txt -n 30

###PGS CATALOG SCORING FILE - see https://www.pgscatalog.org/downloads/#dl_ftp_scoring for additional information
#format_version=2.0
##POLYGENIC SCORE (PGS) INFORMATION
#pgs_id=PGS002724
#pgs_name=GIGASTROKE_iPGS_EUR
#trait_reported=Ischemic stroke
#trait_mapped=stroke|Ischemic stroke
#trait_efo=EFO_0000712|HP_0002140
#genome_build=GRCh37
#variants_number=1213574
#weight_type=NR
##SOURCE INFORMATION
#pgp_id=PGP000333
#citation=Mishra A et al. Nature (2022). doi:10.1038/s41586-022-05165-3
chr_name	chr_position	effect_allele	other_allele	effect_weight
1	752721	G	A	50.2009138795063
1	754182	G	A	141.073654032741
1	760912	T	C	180.556536852976
1	768448	A	G	-74.6438253333578
1	779322	G	A	-137.02495892717
1	838555	A	C	-128.635559661359
1	846808	T	C	-103.154370468722
1	853954	A	C	-128.44445344713
1	854250	G	A	-67.2278476132285
1	861808	G	A	327.215410265433
1	863124	T	G	403.185761038646
1	864938	A	G	-173.628855846674
1	870645	C	T	-80.5551733833478
1	873558	T	G	-261.898526036879
1	879317	T	C	14.

In [14]:
model_PGS002724 = hl.import_table('prs-models/PGS002724.txt', impute=True, force=True, comment='#')

2022-11-23 12:56:23.980 Hail: INFO: Reading table to impute column types
2022-11-23 12:56:31.052 Hail: INFO: Finished type imputation        (0 + 1) / 1]
  Loading field 'chr_name' as type int32 (imputed)
  Loading field 'chr_position' as type int32 (imputed)
  Loading field 'effect_allele' as type str (imputed)
  Loading field 'other_allele' as type str (imputed)
  Loading field 'effect_weight' as type float64 (imputed)


In [15]:
model_PGS002725 = hl.import_table('prs-models/PGS002725.txt', impute=True, force=True, comment='#')

2022-11-23 12:57:00.648 Hail: INFO: Reading table to impute column types
2022-11-23 12:57:16.612 Hail: INFO: Finished type imputation        (1 + 1) / 2]
  Loading field 'chr_name' as type int32 (imputed)
  Loading field 'chr_position' as type int32 (imputed)
  Loading field 'effect_allele' as type str (imputed)
  Loading field 'other_allele' as type str (imputed)
  Loading field 'effect_weight' as type float64 (imputed)


下記のコードは、PRSモデルに含まれるバリアントの個数を表示します。

In [16]:
model_PGS002724.count()

1213574

In [17]:
model_PGS002725.count()

6010730

`1213574` と `6010730` と表示されました。
これは、PRSモデル 002724 と 002725 に含まれるバリアントの個数がそれぞれ 1213574, 6010730 個であることを意味します。

下記のコードは、読み込んだ PRS モデルの最初の 5 行を表示します。

In [18]:
model_PGS002724.show(5)

,,,,
chr_name,chr_position,effect_allele,other_allele,effect_weight
int32,int32,str,str,float64
1,752721,"""G""","""A""",5.02e+01
1,754182,"""G""","""A""",1.41e+02
1,760912,"""T""","""C""",1.81e+02
1,768448,"""A""","""G""",-7.46e+01
1,779322,"""G""","""A""",-1.37e+02


In [19]:
model_PGS002725.show(5)

,,,,
chr_name,chr_position,effect_allele,other_allele,effect_weight
int32,int32,str,str,float64
1,711310,"""A""","""G""",-5.64e-07
1,731718,"""C""","""T""",-3.64e-06
1,732032,"""C""","""A""",-3.54e-06
1,734349,"""C""","""T""",-3.11e-06
1,742990,"""T""","""C""",-5.43e-06


下記のコードは、読み込んだ PRS モデルに `variantID` を追加します。

In [20]:
model_PGS002724 = model_PGS002724.annotate(
    variantID = hl.str(model_PGS002724.chr_name) + ":" + hl.str(model_PGS002724.chr_position) 
)

In [21]:
model_PGS002725 = model_PGS002725.annotate(
    variantID = hl.str(model_PGS002725.chr_name) + ":" + hl.str(model_PGS002725.chr_position) 
)

下記のコードは `variantID` を追加した後の最初の 5 行を表示します。

In [22]:
model_PGS002724.show(5)

,,,,,
chr_name,chr_position,effect_allele,other_allele,effect_weight,variantID
int32,int32,str,str,float64,str
1,752721,"""G""","""A""",5.02e+01,"""1:752721"""
1,754182,"""G""","""A""",1.41e+02,"""1:754182"""
1,760912,"""T""","""C""",1.81e+02,"""1:760912"""
1,768448,"""A""","""G""",-7.46e+01,"""1:768448"""
1,779322,"""G""","""A""",-1.37e+02,"""1:779322"""


In [23]:
model_PGS002725.show(5)

,,,,,
chr_name,chr_position,effect_allele,other_allele,effect_weight,variantID
int32,int32,str,str,float64,str
1,711310,"""A""","""G""",-5.64e-07,"""1:711310"""
1,731718,"""C""","""T""",-3.64e-06,"""1:731718"""
1,732032,"""C""","""A""",-3.54e-06,"""1:732032"""
1,734349,"""C""","""T""",-3.11e-06,"""1:734349"""
1,742990,"""T""","""C""",-5.43e-06,"""1:742990"""


`variantID` のカラムが追加されていることが分かります。

## Step 9-2 ゲノムデータとPRSモデルに共通するバリアントを抽出します
下記のコードは、PRSモデルのバリアント情報を variantID で検索できるようにします。

In [24]:
model_PGS002724 = model_PGS002724.key_by('variantID')

In [25]:
model_PGS002725 = model_PGS002725.key_by('variantID')

In [26]:
mt1_filt.rows().show(5)

+---------------+------------+---------------+-----------+----------+
| locus         | alleles    | rsid          |      qual | filters  |
+---------------+------------+---------------+-----------+----------+
| locus<GRCh37> | array<str> | str           |   float64 | set<str> |
+---------------+------------+---------------+-----------+----------+
| 1:534247      | ["C","T"]  | "rs201475892" | -1.00e+01 | {}       |
| 1:565286      | ["C","T"]  | "rs1578391"   | -1.00e+01 | {}       |
| 1:674211      | ["C","T"]  | "rs546906063" | -1.00e+01 | {}       |
| 1:701299      | ["A","G"]  | "rs553919012" | -1.00e+01 | {}       |
| 1:701625      | ["T","C"]  | "rs576411494" | -1.00e+01 | {}       |
+---------------+------------+---------------+-----------+----------+

+----------------+----------------+----------+------------+
| info.AF        | info.DR2       | info.IMP | variantID  |
+----------------+----------------+----------+------------+
| array<float64> | array<float64> |     bool | str        |
+----------------+----------------+----------+------------+
| [7.80e-03]     | [1.00e+00]     |    False | "1:534247" |
| [9.93e-01]     | [1.00e+00]     |    False | "1:565286" |
| [2.25e-02]     | [4.40e-01]     |     True | "1:674211" |
| [2.57e-02]     | [7.20e-01]     |     True | "1:701299" |
| [2.40e-03]     | [5.60e-01]     |     True | "1:701625" |
+----------------+----------------+----------+------------+
showing top 5 rows

下記のコードは、ゲノムデータと PRS モデルに共通するバリアントを抽出します。

In [27]:
mt_match_PGS002724 = mt1_filt.annotate_rows(**model_PGS002724[mt1_filt.variantID])
mt_match_PGS002724 = mt_match_PGS002724.filter_rows(hl.is_defined(mt_match_PGS002724.effect_weight))

In [28]:
mt_match_PGS002725 = mt1_filt.annotate_rows(**model_PGS002725[mt1_filt.variantID])
mt_match_PGS002725 = mt_match_PGS002725.filter_rows(hl.is_defined(mt_match_PGS002725.effect_weight))

In [29]:
mt_match_PGS002724.rows().show(5)

2022-11-23 13:05:08.949 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-23 13:05:16.082 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-23 13:11:19.282 Hail: INFO: Ordering unsorted dataset with network shuffle


+---------------+------------+--------------+-----------+----------+
| locus         | alleles    | rsid         |      qual | filters  |
+---------------+------------+--------------+-----------+----------+
| locus<GRCh37> | array<str> | str          |   float64 | set<str> |
+---------------+------------+--------------+-----------+----------+
| 1:752721      | ["A","G"]  | "rs3131972"  | -1.00e+01 | {}       |
| 1:754182      | ["A","G"]  | "rs3131969"  | -1.00e+01 | {}       |
| 1:760912      | ["C","T"]  | "rs1048488"  | -1.00e+01 | {}       |
| 1:768448      | ["G","A"]  | "rs12562034" | -1.00e+01 | {}       |
| 1:779322      | ["A","G"]  | "rs4040617"  | -1.00e+01 | {}       |
+---------------+------------+--------------+-----------+----------+

+----------------+----------------+----------+------------+----------+
| info.AF        | info.DR2       | info.IMP | variantID  | chr_name |
+----------------+----------------+----------+------------+----------+
| array<float64> | array<float64> |     bool | str        |    int32 |
+----------------+----------------+----------+------------+----------+
| [6.73e-01]     | [1.00e+00]     |    False | "1:752721" |        1 |
| [7.01e-01]     | [9.90e-01]     |     True | "1:754182" |        1 |
| [7.49e-01]     | [9.80e-01]     |     True | "1:760912" |        1 |
| [1.57e-01]     | [1.00e+00]     |    False | "1:768448" |        1 |
| [2.11e-01]     | [1.00e+00]     |    False | "1:779322" |        1 |
+----------------+----------------+----------+------------+----------+

+--------------+---------------+--------------+---------------+
| chr_position | effect_allele | other_allele | effect_weight |
+--------------+---------------+--------------+---------------+
|        int32 | str           | str          |       float64 |
+--------------+---------------+--------------+---------------+
|       752721 | "G"           | "A"          |      5.02e+01 |
|       754182 | "G"           | "A"          |      1.41e+02 |
|       760912 | "T"           | "C"          |      1.81e+02 |
|       768448 | "A"           | "G"          |     -7.46e+01 |
|       779322 | "G"           | "A"          |     -1.37e+02 |
+--------------+---------------+--------------+---------------+
showing top 5 rows

In [27]:
model_PGS002724.filter(model_PGS002724.chr_name==1).count()

99864

In [37]:
model_PGS002725.filter(model_PGS002725.chr_name==1).count()

475080

今後のステップの実行時間を短縮するため(ここを第2のチェックポイントとするため)、一旦抽出したゲノムデータを保存しておきます。

下記のコードは、抽出したゲノムデータを `outputs/chrAll.filtered.matched_PGS002724.mt`, `outputs/chrAll.filtered.matched_PGS002725.mt` に保存します。


In [29]:
mt_match_PGS002724.write('outputs/chrAll.filtered.matched_PGS002724.mt', overwrite=True)


2022-11-21 16:25:48.517 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-21 16:25:53.704 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-21 16:34:47.212 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-21 16:51:37.284 Hail: INFO: wrote matrix table with 1211967 rows and 2318 columns in 89 partitions to outputs/chrAll.filtered.matched_PGS002724.mt


In [38]:
mt_match_PGS002725.write('outputs/chrAll.filtered.matched_PGS002725.mt', overwrite=True)


2022-11-21 21:56:42.035 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-21 21:56:54.561 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-21 22:13:20.912 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-21 22:52:14.176 Hail: INFO: wrote matrix table with 5930286 rows and 2318 columns in 89 partitions to outputs/chrAll.filtered.matched_PGS002725.mt


In [30]:
len(dict(mt_match_PGS002724.aggregate_rows(hl.agg.counter(mt_match_PGS002724.variantID))))

2022-11-23 13:20:39.449 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-23 13:20:51.962 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-23 13:26:39.054 Hail: INFO: Ordering unsorted dataset with network shuffle


1211916

In [31]:
len(dict(mt_match_PGS002725.aggregate_rows(hl.agg.counter(mt_match_PGS002725.variantID))))

2022-11-23 14:05:53.594 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-23 14:06:12.039 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-23 14:14:15.707 Hail: INFO: Ordering unsorted dataset with network shuffle


5929754

## Step 9-3 PRSを計算して保存します
下記のコードは、ゲノムデータのアリル情報とPRSモデルのアリル情報を比較し、合致しているかをチェックします。

In [32]:
flip_PGS002724 = hl.case().when( 
    (mt_match_PGS002724.effect_allele == mt_match_PGS002724.alleles[0])
    & (mt_match_PGS002724.other_allele == mt_match_PGS002724.alleles[1]), True ).when( 
    (mt_match_PGS002724.effect_allele == mt_match_PGS002724.alleles[1])
    & (mt_match_PGS002724.other_allele == mt_match_PGS002724.alleles[0]), False ).or_missing()

In [33]:
flip_PGS002725 = hl.case().when( 
    (mt_match_PGS002725.effect_allele == mt_match_PGS002725.alleles[0]) 
    & (mt_match_PGS002725.other_allele == mt_match_PGS002725.alleles[1]), True ).when( 
    (mt_match_PGS002725.effect_allele == mt_match_PGS002725.alleles[1])
    & (mt_match_PGS002725.other_allele == mt_match_PGS002725.alleles[0]), False ).or_missing()

In [34]:
mt_match_PGS002724 = mt_match_PGS002724.annotate_rows(flip=flip_PGS002724)

In [35]:
mt_match_PGS002725 = mt_match_PGS002725.annotate_rows(flip=flip_PGS002725)

下記のコードは、各々のバリアントについて研究対象者の持っているアリル数（`mt_match_PGS002724.DS`）とバリアントの重み（`mt_match_PGS002724.effect_weight`）を掛け合わせて、ゲノムデータとPRSモデルの共通するバリアントについて足し合わせます。 これにより、PRSを計算することができます。

In [36]:
prs_PGS002724 = hl.agg.sum(hl.float64(mt_match_PGS002724.effect_weight) * 
                    hl.if_else( mt_match_PGS002724.flip, 
                                2 - mt_match_PGS002724.DS.first(),
                                mt_match_PGS002724.DS.first()))

In [38]:
mt_match_PGS002724 = mt_match_PGS002724.annotate_cols(prs=prs_PGS002724)

PGS002725 についても同様のことを行ってみましょう。

In [37]:
prs_PGS002725 = hl.agg.sum(hl.float64(mt_match_PGS002725.effect_weight) * 
                    hl.if_else( mt_match_PGS002725.flip, 
                                2 - mt_match_PGS002725.DS.first(),
                                mt_match_PGS002725.DS.first()))

In [39]:
mt_match_PGS002725 = mt_match_PGS002725.annotate_cols(prs=prs_PGS002725)

下記のコードは、PRSの値を表示します。

In [40]:
mt_match_PGS002724.cols().show(5)

2022-11-23 16:24:35.335 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'
2022-11-23 16:29:55.700 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-23 16:30:04.670 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-23 16:36:22.299 Hail: INFO: Ordering unsorted dataset with network shuffle


,
s,prs
str,float64
"""_HG00096""",-2.35e+06
"""_HG00097""",-2.39e+06
"""_HG00098""",-2.27e+06
"""_HG00099""",-2.39e+06
"""_HG00100""",-2.25e+06


In [41]:
mt_match_PGS002725.cols().show(5)

2022-11-23 16:53:43.532 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-23 16:54:00.362 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-23 17:01:50.156 Hail: INFO: Ordering unsorted dataset with network shuffle


,
s,prs
str,float64
"""_HG00096""",-1.73e+00
"""_HG00097""",-1.48e+00
"""_HG00098""",-1.82e+00
"""_HG00099""",-2.21e+00
"""_HG00100""",-1.47e+00


下記のコードは、PRSの分布を表示します。

In [42]:
p = hl.plot.histogram(mt_match_PGS002724.prs, title="PRS Histogram", legend="PGS002724", bins=20)
show(p)

2022-11-23 17:18:12.181 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-23 17:18:20.554 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-23 17:23:52.106 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-23 17:36:20.617 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-23 17:36:26.549 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-23 17:42:27.683 Hail: INFO: Ordering unsorted dataset with network shuffle


下記のコードは、PRSの計算結果を chrAll.filtered.PGS002724.PRS.txt ファイルに保存します。

In [43]:
mt_match_PGS002724.cols().export('chrAll.filtered.PGS002724.PRS.txt')

2022-11-23 17:57:54.529 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-23 17:58:03.476 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-23 18:03:44.519 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-23 18:11:38.831 Hail: INFO: Coerced sorted dataset=======>(88 + 1) / 89]
2022-11-23 18:11:40.349 Hail: INFO: merging 17 files totalling 47.5K...
2022-11-23 18:11:40.442 Hail: INFO: while writing:
    chrAll.filtered.PGS002724.PRS.txt
  merge time: 91.440ms


PGS002725 についても同様のことを行ってみましょう。

In [44]:
p = hl.plot.histogram(mt_match_PGS002725.prs, title="PRS Histogram", legend="PGS002725", bins=20)
show(p)

2022-11-23 18:34:36.295 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-23 18:34:57.684 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-23 18:43:45.743 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-23 18:59:19.136 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-23 18:59:31.370 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-23 19:08:03.664 Hail: INFO: Ordering unsorted dataset with network shuffle


In [45]:
mt_match_PGS002725.cols().export('chrAll.filtered.PGS002725.PRS.txt')

2022-11-23 19:34:31.787 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-23 19:34:49.417 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-23 19:43:15.359 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-23 19:54:13.539 Hail: INFO: Coerced sorted dataset========(89 + 0) / 89]
2022-11-23 19:54:13.800 Hail: INFO: merging 17 files totalling 47.5K...
2022-11-23 19:54:13.828 Hail: INFO: while writing:
    chrAll.filtered.PGS002725.PRS.txt
  merge time: 26.715ms


## Step 10 PRSのスコア値を比較します
ここまでで 2 つの脳梗塞PRSモデルを用いて、PRSスコア値を計算しました。

このチュートリアルの最後に、そのPRSスコア値を比較してみます。

下記のコードは、PRSモデル PGS002724 を用いて計算した PRS スコア値を読み込みます。

In [46]:
prs_PGS002724 = hl.import_table('chrAll.filtered.PGS002724.PRS.txt', impute=True, force=True)

2022-11-23 20:19:37.892 Hail: INFO: Reading table to impute column types
2022-11-23 20:19:38.596 Hail: INFO: Finished type imputation
  Loading field 's' as type str (imputed)
  Loading field 'prs' as type float64 (imputed)


In [47]:
prs_PGS002725 = hl.import_table('chrAll.filtered.PGS002725.PRS.txt', impute=True, force=True)

2022-11-23 20:20:10.640 Hail: INFO: Reading table to impute column types
2022-11-23 20:20:11.192 Hail: INFO: Finished type imputation
  Loading field 's' as type str (imputed)
  Loading field 'prs' as type float64 (imputed)


下記のコードは、PRS スコア値を subject ID (変数名 `s`) で検索できるようにします。

In [48]:
prs_PGS002724 = prs_PGS002724.key_by('s')

In [49]:
prs_PGS002725 = prs_PGS002725.key_by('s')

下記のコードは、2 種類の PRS スコア値を subject ID (変数名 s) で突合し、データマージを行います。

In [50]:
prs_merge = prs_PGS002724.rename({'s':'subjectID', 'prs':'PGS002724'})

In [51]:
prs_merge = prs_merge.annotate(PGS002725 = prs_PGS002725[prs_merge.subjectID].prs)

下記のコードは、データマージした結果を表示します。

In [52]:
prs_merge.show(5)

2022-11-23 20:24:34.655 Hail: INFO: Coerced sorted dataset
2022-11-23 20:24:35.058 Hail: INFO: Coerced sorted dataset


,,
subjectID,PGS002724,PGS002725
str,float64,float64
"""_HG00096""",-2.35e+06,-1.73e+00
"""_HG00097""",-2.39e+06,-1.48e+00
"""_HG00098""",-2.27e+06,-1.82e+00
"""_HG00099""",-2.39e+06,-2.21e+00
"""_HG00100""",-2.25e+06,-1.47e+00


下記のコードは、`PGS002724` と `PGS002725` のスコア値をプロットします。

In [53]:
p = hl.plot.scatter(prs_merge.PGS002724, prs_merge.PGS002725, xlabel="PGS002724", ylabel="PGS002725")
show(p)

2022-11-23 20:25:35.986 Hail: INFO: Coerced sorted dataset
2022-11-23 20:25:36.310 Hail: INFO: Coerced sorted dataset


下記のコードは、PRS スコア値の相関係数を計算します。

`to_pandas()`関数を用いることで、`hail`特有のデータタイプから、`python` でよく使われる `pandas` ライブラリのデータフレームへと変換することができます。
そうすることで、`hail`の関数だけでなく、`python` の様々な機能を使って分析することが可能になります。

In [54]:
prs_merge_pandas = prs_merge.to_pandas()

2022-11-23 20:26:30.449 Hail: INFO: Coerced sorted dataset
2022-11-23 20:26:30.729 Hail: INFO: Coerced sorted dataset


In [55]:
print(prs_merge_pandas.corr())

           PGS002724  PGS002725
PGS002724   1.000000   0.168966
PGS002725   0.168966   1.000000


この結果から、次のことが分かります。

- PGS002724のスコア値と PGS002725のスコア値の相関係数は　0.168966

### 以上でこのチュートリアルは終了です